In [ ]:
# 개발 환경: 윈도우, 맥, 구글 코랩

# Python: 3.10.12

# 라이브러리 버전:
# Pandas 버전: 1.5.3
# NumPy 버전: 1.23.5
# Scikit-Learn 버전: 1.2.2
# PyTorch 버전: 2.0.1+cu118
# Tqdm 버전: 4.66.1

In [ ]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [ ]:
#PyTorch에서 GPU를 사용할 수 있는지 확인하고, GPU가 사용 가능한 경우 GPU를 사용하고, 그렇지 않으면 CPU를 사용하는 디바이스(device)를 설정
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
# 다양한 설정 및 하이퍼파라미터를 저장

CFG = {
    'TRAIN_WINDOW_SIZE':120, # 120일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':15,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':2024,
    'SEED':41
}

In [ ]:

# 실험의 재현성을 위해 무작위 시드(seed)를 설정하는 함수
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오기

In [ ]:
#Google Colab 환경에서 Google Drive를 마운트하고 '/content/drive' 경로에 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#주어진 파일 경로에서 CSV 파일을 읽어오고, 데이터프레임에서 'ID'와 '제품' 열을 삭제
train_data = pd.read_csv('/content/drive/MyDrive/LgAimers/2023_AI/train.csv').drop(columns=['ID', '제품'])

### 데이터 전처리

In [ ]:
train_data.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train_data.columns[4:]
# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)
# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges = max_values - min_values
ranges[ranges == 0] = 1
# min-max scaling 수행
train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()

In [ ]:
# 'LabelEncoder' 객체를 생성하여 범주형 데이터를 라벨 인코딩합니다.
label_encoder = LabelEncoder()


# 라벨 인코딩을 수행할 대상 열을 'categorical_columns' 변수에 정의합니다.
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

# 각 범주형 열에 대해 라벨 인코딩을 수행합니다.
for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [ ]:
# 'train_data' 데이터프레임에서 '브랜드' 열을 삭제합니다.
train_data = train_data.drop(['브랜드'], axis = 1)
# 수정된 데이터프레임의 처음 몇 개의 행을 표시합니다.
train_data.head()

,대분류,중분류,소분류,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,6,37,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,2,7,43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0
2,2,7,43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,2,7,43,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [ ]:
# 학습용 데이터와 예측용 데이터를 생성하는 함수

def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
    data : 일별 판매량
    train_size : 학습에 활용할 기간
    predict_size : 추론할 기간
    '''
    # 데이터의 행 수를 가져옵니다.
    num_rows = len(data)
    window_size = train_size + predict_size
    # 입력 데이터와 타겟 데이터를 저장할 빈 배열을 생성합니다.
    input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :3]) + 1))
    target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    # 데이터를 반복하며 윈도우를 만들고 입력 및 타겟 데이터에 저장합니다.
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :3])
        sales_data = np.array(data.iloc[i, 3:])

        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
            target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]

    return input_data, target_data

In [ ]:

# 평가 데이터를 추론하기 위한 입력 데이터를 생성하는데 사용되는 함수

def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
    data : 일별 판매량
    train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
    '''
     # 데이터의 행 수를 가져옵니다.
    num_rows = len(data)
    # 입력 데이터를 저장할 빈 배열을 생성합니다.
    input_data = np.empty((num_rows, train_size, len(data.iloc[0, :3]) + 1))
    # 데이터를 반복하며 입력 데이터를 생성합니다.
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :3])
        sales_data = np.array(data.iloc[i, -train_size:])

        window = sales_data[-train_size : ]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data[i] = temp_data

    return input_data

In [ ]:
# 'make_train_data' 함수를 사용하여 학습용 입력 데이터와 타겟 데이터를 생성합니다
train_input, train_target = make_train_data(train_data)
# 'make_predict_data' 함수를 사용하여 테스트(평가)용 입력 데이터를 생성합니다.
test_input = make_predict_data(train_data)

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [ ]:
# Train / Validation Split
# 학습 데이터의 총 길이를 가져옵니다.
data_len = len(train_input)

# 데이터를 학습 세트와 검증 세트로 나눕니다.
# 일반적으로 데이터의 일부를 검증 세트로 분할하여 모델의 성능을 평가합니다.
# 여기서는 데이터의 약 30%를 검증 세트로 사용합니다.
val_input = train_input[-int(data_len*0.3):]    # 데이터의 뒷부분을 검증 입력 데이터로 선택
val_target = train_target[-int(data_len*0.3):]  # 검증 타겟 데이터로 선택
train_input = train_input[:-int(data_len*0.3)]  # 나머지를 학습 입력 데이터로 선택
train_target = train_target[:-int(data_len*0.3)]  # 학습 타겟 데이터로 선택

In [ ]:
# 학습용 입력 데이터의 형태(shape)를 출력합니다.
# 학습용 타겟 데이터의 형태(shape)를 출력합니다.
# 검증용 입력 데이터의 형태(shape)를 출력합니다.
# 검증용 타겟 데이터의 형태(shape)를 출력합니다.
# 테스트(평가)용 입력 데이터의 형태(shape)를 출력합니다.

train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((3581606, 120, 4),
 (3581606, 21),
 (1534974, 120, 4),
 (1534974, 21),
 (15890, 120, 4))

### Custom Dataset

In [ ]:

# 사용자 정의 데이터셋을 생성하기 위한 'CustomDataset' 클래스를 정의합니다.
class CustomDataset(Dataset):
   # 생성자 메서드에서 입력 데이터(X)와 타겟 데이터(Y)를 초기화합니다.
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y

    def __getitem__(self, index):
        if self.Y is not None:
            # 데이터셋에서 지정된 인덱스(index)에 해당하는 샘플을 반환합니다.
            # 입력 데이터와 타겟 데이터를 묶어서 반환합니다.
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
             # 타겟 데이터가 없는 경우, 입력 데이터만 반환합니다.
        return torch.Tensor(self.X[index])

    def __len__(self):
        # 데이터셋의 전체 길이를 반환합니다.
        return len(self.X)

In [ ]:
# PyTorch의 DataLoader를 사용하여 학습 데이터와 검증 데이터를 미니 배치로 로딩하는 작업을 수행

train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0, pin_memory=True)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0, pin_memory=True)

### 모델 선언

In [ ]:
# 어텐션 메커니즘을 구현한 Dot Product Attention 클래스
class DotProductAttention(nn.Module):
    def __init__(self):
        super(DotProductAttention, self).__init__()

    def forward(self, query, key, value):
        # 어텐션 스코어 계산
        scores = torch.bmm(query, key.transpose(1, 2))

        # 어텐션 가중치 계산
        attn_weights = torch.nn.functional.softmax(scores, dim=-1)

        # 가중 평균을 이용하여 컨텍스트 벡터 계산
        context = torch.bmm(attn_weights, value)
        return context, attn_weights

# 메인 모델 클래스
class Model(nn.Module):
    def __init__(self, input_size=4, hidden_size=1024, output_size=CFG['PREDICT_SIZE']):
        super(Model, self).__init__()

        self.hidden_size = hidden_size

        # LSTM 레이어 정의
        self.lstm_1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.lstm_2 = nn.LSTM(hidden_size, hidden_size, batch_first=True)

        # GRU 레이어 정의
        self.gru = nn.GRU(hidden_size, hidden_size, num_layers=1, batch_first=True)

        # 어텐션 레이어 정의
        self.attention = DotProductAttention()

        # 드롭아웃 레이어 정의
        self.lstm_dropout = nn.Dropout(0.3)
        self.gru_dropout = nn.Dropout(0.2)

        # 완전 연결 레이어 정의
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.GELU(),
            nn.Dropout(0.2),
            nn.Linear(hidden_size // 2, output_size)
        )
        self.actv = nn.GELU()

    def init_hidden(self, batch_size, device):
        return (torch.zeros(1, batch_size, self.hidden_size, device=device),
                torch.zeros(1, batch_size, self.hidden_size, device=device))

    def forward(self, x):
        batch_size = x.size(0)
        seq_length = x.size(1)

        # 초기 hidden state 초기화
        hidden_1 = self.init_hidden(batch_size, x.device)
        hidden_2 = self.init_hidden(batch_size, x.device)
        hidden_3 = self.init_hidden(batch_size, x.device)

        # 첫 번째 LSTM 레이어 진행
        lstm_out_1, _ = self.lstm_1(x, hidden_1)
        lstm_out_1 = self.lstm_dropout(lstm_out_1)
        lstm_out_1 = self.actv(lstm_out_1)

        # 어텐션 메커니즘을 이용하여 컨텍스트 벡터 계산
        context_1, _ = self.attention(lstm_out_1, lstm_out_1, lstm_out_1)

        # 두 번째 LSTM 레이어 진행
        lstm_out_2, _ = self.lstm_2(lstm_out_1, hidden_2)
        lstm_out_2 = self.lstm_dropout(lstm_out_2)
        lstm_out_2 = self.actv(lstm_out_2)

        # 어텐션 메커니즘을 이용하여 컨텍스트 벡터 계산
        context_2, _ = self.attention(lstm_out_2, lstm_out_2, lstm_out_2)

        # 스킵 연결을 이용하여 어텐션 결과와 원래 입력을 합침
        skip_attention = lstm_out_1 + context_2

        # GRU 레이어 진행
        gru_out, _ = self.gru(context_1, hidden_3[0])
        gru_out = self.gru_dropout(gru_out)

        # 어텐션 메커니즘을 이용하여 컨텍스트 벡터 계산
        context_3, _ = self.attention(gru_out, gru_out, gru_out)

        # 어텐션 결과와 스킵 연결 결과를 더하여 최종 결과 생성
        attended_output = skip_attention[:, -1, :] + context_3[:, -1, :]

        # 완전 연결 레이어를 통과시켜 최종 출력 생성
        output = self.actv(self.fc(attended_output))

        return output.squeeze(1)

### 모델 학습

In [ ]:
# 주어진 모델을 학습하고 검증 데이터를 사용하여 모델을 평가하며, 가장 좋은 모델을 선택하여 저장하는 역할을 하는 함수

def train(model, optimizer, scheduler, train_loader, val_loader, device, patience):
     # 모델을 지정한 디바이스(GPU 또는 CPU)로 이동합니다.
    model.to(device)
     # 손실 함수(Mean Squared Error)를 지정한 디바이스로 이동합니다.
    criterion = nn.MSELoss().to(device)
     # 가장 낮은 검증 손실을 추적하기 위한 변수 초기화
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        # 미니 배치별로 학습을 수행합니다.
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()
            optimizer.step()

            train_loss.append(loss.item())

        # 검증 데이터를 사용하여 검증 손실을 계산합니다.
        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        # 현재 검증 손실이 최고 손실보다 낮다면 모델을 저장하고 손실 업데이트
        if val_loss < best_loss:
            best_loss = val_loss
            best_model = model
            early_stopping_counter = 0
            print('Model Saved')

            # 모델 파라미터 저장
            model_path = os.path.join('/content/drive/MyDrive/LgAimers/2023_AI/Lg_Aimers_parameter.pth')
            torch.save(model.state_dict(), model_path)
            print(f'Model parameters saved at {model_path}')
        else:
            early_stopping_counter += 1
            if early_stopping_counter >= patience:
                print(f'Early stopping at epoch {epoch}')
                break
        # 스케줄러를 업데이트합니다.
        scheduler.step()
    return best_model

In [ ]:
def validation(model, val_loader, criterion, device):
   # 모델을 평가 모드로 설정합니다.
    model.eval()
    # 검증 손실을 저장할 리스트 초기화
    val_loss = []

    with torch.no_grad():
       # 미니 배치별로 검증을 수행합니다.
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            # 모델에 입력 데이터를 전달하여 예측을 수행합니다.
            output = model(X)
             # 손실 함수를 사용하여 손실을 계산합니다.
            loss = criterion(output, Y)
            # 검증 손실을 리스트에 추가합니다.
            val_loss.append(loss.item())
    # 검증 손실의 평균을 반환합니다.
    return np.mean(val_loss)

## Run !!

In [ ]:

# 메모리를 수동으로 해제하는 작업을 수행
import gc
gc.collect()

0

In [ ]:
#현재 GPU 메모리에서 사용하지 않는 자원을 해제하고 메모리 사용량을 최소화하는데 필요한 코드

for i in range(10000):
    torch.cuda.empty_cache()

In [ ]:

# 새로운 모델을 생성합니다.
model = Model()
# AdamW 옵티마이저를 생성하고 모델의 파라미터를 최적화합니다.
# 학습률은 'CFG["LEARNING_RATE"]'로 설정됩니다.
optimizer = torch.optim.AdamW(params = model.parameters(), lr = CFG["LEARNING_RATE"])

# Cosine Annealing with Warm Restarts 스케줄러를 생성합니다.
# 'T_0'는 초기 주기, 'T_mult'는 주기를 어떻게 확장할지, 'eta_min'은 최소 학습률을 설정합니다.
scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=2,
                                                                 T_mult=1, eta_min=0.000007) # 변경 사항

# 모델을 학습하는 'train' 함수를 호출합니다.
# 'patience'는 조기 종료를 위한 인내 값입니다.
infer_model = train(model, optimizer, scheduler, train_loader, val_loader, device, patience=3)

  0%|          | 0/1770 [00:00<?, ?it/s]

  0%|          | 0/759 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.01951] Val Loss : [0.01724]
Model Saved
Model parameters saved at /content/drive/MyDrive/LgAimers/2023_AI/best_model.pth


  0%|          | 0/1770 [00:00<?, ?it/s]

  0%|          | 0/759 [00:00<?, ?it/s]

...

  0%|          | 0/759 [00:00<?, ?it/s]

Epoch : [13] Train Loss : [0.01614] Val Loss : [0.01674]


  0%|          | 0/1770 [00:00<?, ?it/s]

  0%|          | 0/759 [00:00<?, ?it/s]

Epoch : [14] Train Loss : [0.01597] Val Loss : [0.01702]
Early stopping at epoch 14


## 모델 추론

In [ ]:
#CUDA GPU 메모리를 비워주는 역할
for i in range(10000):
    torch.cuda.empty_cache()

In [ ]:
#현재 GPU 메모리에서 사용하지 않는 자원을 해제하는 역할

torch.cuda.empty_cache()

In [ ]:
# 학습된 모델을 사용하여 테스트 데이터에 대한 예측을 수행하기 위한 작업
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []

    with torch.no_grad():
       # 테스트 데이터 미니 배치별로 추론을 수행합니다.
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            # 모델에 입력 데이터를 전달하여 예측을 수행합니다.
            output = model(X)

            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
             # 예측 결과를 리스트에 추가합니다.
            predictions.extend(output)
     # 모든 예측 결과를 하나의 numpy 배열로 반환합니다.
    return np.array(predictions)

In [ ]:
pred = inference(infer_model, test_loader, device)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [ ]:
pred.shape

(15890, 21)

## Submission

In [ ]:
# Google Drive에서 샘플 제출 파일을 읽어와 데이터를 확인하는 작업을 수행
submit = pd.read_csv('/content/drive/MyDrive/LgAimers/2023_AI/sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 예측 결과를 제출 파일의 적절한 열에 대입하여 제출 파일을 업데이트하는 작업을 수행
submit.iloc[:, 1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,0
1,1,1,1,1,1,1,1,1,1,2,...,1,1,1,2,2,2,2,1,1,1
2,2,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,0,1,1,1,1
3,3,0,0,0,0,0,1,1,1,1,...,1,1,1,1,1,1,1,2,2,2
4,4,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [ ]:
# '2023-04-05' 열의 값들을 카운트하고, 각 고유 값의 등장 횟수를 계산하여 반환
submit['2023-04-05'].value_counts()

0      9806
1      1342
2       627
3       423
4       336
       ... 
688       1
885       1
598       1
355       1
137       1
Name: 2023-04-05, Length: 356, dtype: int64

In [ ]:
# DataFrame을 CSV 파일로 저장하는 작업을 수행
submit.to_csv('./Lg_Aimers_submission.csv', index=False)